# QUICK PLAY ROUND
### Create a tool that summarizes an online article or blog post.

In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [5]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [6]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

# Let's make a quick call to a Frontier model to get started, as a preview!

In [7]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! It's great to hear from you. How can I assist you today?


## Scrape the website

In [12]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
# Let's try one out. Change the website and add print statements to follow along.

post = Website("https://zandersgroup.com/en/ifrs9-annual-report-study")
print(post.title)
print("------")
print(post.text)

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [18]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of an article \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [19]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at an article titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [20]:
system_prompt

'You are an assistant that analyzes the contents of an article and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'

In [22]:
print(user_prompt_for(post))

You are looking at an article titled IFRS 9 - Annual Report Study - Zanders
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

ENG
English
Login
SOLUTIONS
By Role
CEO & Board
CFO
CRO & Risk Manager
Corporate Treasurer
Financial Controller
Political Decision-maker
By Need – Treasury
Benchmark my treasury
Transform my treasury
Digitize my treasury
Manage my treasury
By Need – Risk
Assess my risk
Mitigate my risk
Manage my risk
Optimize my upside
By Need – Technology
Benchmark my technology
Optimize my process
Implement my systems
Manage my technology
EXPERTISE
Treasury
Treasury Strategy
Strategic Benchmarking
M&A integration & divestments
Organization & governance
Treasury Operations
Cash management & cashflow forecasting
Inhouse bank & payments
Managed services
Treasury Transformation
Technology implementation
Technology selection
Treasury Digitalization
Blockchain & DeF

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [23]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [24]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, let me dust off my calculator for this one! It's 4. Shocking, I know!


## And now let's build useful messages for GPT-4o-mini, using a function

In [25]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
# Try this out, and then try for a few more websites

messages_for(post)

## Time to bring it together - the API for OpenAI is very simple!

In [27]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [28]:
summarize("https://zandersgroup.com/en/ifrs9-annual-report-study")

'# Summary of IFRS 9 - Annual Report Study - Zanders\n\nZanders conducted an annual report study analyzing the IFRS 9 results across four European markets: the Nordics, the Netherlands, DACH (Germany, Austria, and Switzerland), and the UK. The study identifies both similarities and differences in trends for Expected Credit Losses (ECLs) across these regions from 2020 to 2023, influenced by macroeconomic factors like COVID-19 and the war in Ukraine, while highlighting significant cultural differences in modeling practices.\n\n## Key Findings by Region:\n\n### Nordics\n- A slight decrease in overall coverage ratios from 2020 to 2023 was noted, with OP Bank showing an increase due to improvements in stage 2 ratios.\n- Variation in Stage 3 coverage ratios among banks, highlighting different modeling approaches.\n\n### Netherlands\n- A slight decline in ECL results correlated with improved macroeconomic conditions and a reversal of COVID-related manual overlays.\n- Larger banks saw coverage

In [29]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [30]:
display_summary("https://zandersgroup.com/en/ifrs9-annual-report-study")

# Summary of IFRS 9 - Annual Report Study - Zanders

Zanders has published an annual report study focusing on the impacts of IFRS 9 across four European markets: the Nordics, the Netherlands, DACH (Germany, Austria, Switzerland), and the UK. The analysis highlights both common trends and significant cultural differences in how Expected Credit Losses (ECLs) are modeled across these regions, influenced by recent systemic macroeconomic events like Covid-19 and the war in Ukraine.

## Key Findings by Region:

### Nordics:
- Coverage ratios for banks like OP showed a slight decline, except for OP, which increased its Stage 2 coverage ratio.
- The difference in Stage 2 ratios across banks was notable, with one bank (Jyske Bank) showing considerable volatility.

### Netherlands:
- A slight decrease in overall ECL results was observed, attributed to an improved macroeconomic outlook and reversed COVID-19 overlays.
- Larger banks reported lower coverage ratios compared to smaller banks.

### DACH:
- A stable coverage ratio trend was reported until 2022, followed by an increase in 2023.
- Significant discrepancies were noted, especially with RBI having markedly higher coverage ratios.

### UK:
- Most banks reported a decrease in coverage ratios during 2020-2021, followed by stabilization in subsequent years.
- Monzo exhibited a high coverage ratio despite significant expansion in lending.

### Overall Trends:
- The general trend showed a decrease in ECL coverage ratios from 2020 to 2021, with DACH exhibiting an increase in 2023.
- A need for alignment in modeling ECLs across European regions is emphasized given the cultural differences observed.

Zanders aims to leverage its presence in these regions to assist banks in enhancing their IFRS 9 models, ensuring accurate ECL estimates through independent validations or new model developments.

For further information, readers are invited to contact the author, Kasper Wijshoff.

In [ ]:
# Step 1: Create your prompts

system_prompt = "something here"
user_prompt = """
    Lots of text
    Can be pasted here
"""

# Step 2: Make the messages list

messages = [] # fill this in

# Step 3: Call OpenAI

response =

# Step 4: print the result

print(